# Are Tesla's Shares Overpriced

On 1st May 2020 Elon Musk tweeted that "Tesla stock price is too high imo." Whilst Elon Musk is known for is controversial tweets, let's look at Tesla's share price using popular metrics to find out if he is correct.

Whilst the aim of this project is to answer the above question. We are using this to develop the following skills:

- Web scraping
- Accessing data via an API
- Data visualisation with Plotly 


In [1]:
import datetime as dt
import numpy as np
import pandas as pd

## How Will We Determine If Telsas Stock Price Is Overvalued

We're going to use ratios from investopedias article on determining over or undervalued stocks found here:

https://www.investopedia.com/articles/investing/101316/how-tell-if-stock-overvalued-or-undervalued.asp

The Ratios we are interested in are:

- P/E, P/B, and EV/EBITDA Ratios 
- Price-to-Sales ratio (P/S)
- Price-to-Dividend (P/D)
- Enterprise Value-to-Sales (EV/S) Ratio 
- Price/Earnings to Growth (PEG) Ratio

We will first start by accessing the data using the financial modeling prep api which can be found here:

https://financialmodelingprep.com/developer/docs/#Company-Financial-Ratios




In [2]:
import json

def financialratios(quote):
    fr = requests.get(f"https://financialmodelingprep.com/api/v3/financial-ratios/{quote}")
    fr = fr.json()
    #Get data from api and using a for loop get the ratios for our selected areas and assigning them to a value.
    frames = []
    areas = ["valuation", "profitability", "operating", "liquidity", "debt"]
    fin_ratios = ["investmentValuationRatios", "profitabilityIndicatorRatios", "operatingPerformanceRatios", "liquidityMeasurementRatios", "debtRatios"]
    
    for area, ratio in zip(areas, fin_ratios):
        area = fr['ratios'][0][ratio]
        area = pd.DataFrame(list(area.items()),columns=['Ratio', quote])
        frames.append(area)
    
    df = pd.concat(frames)
    
    our_ratios = ["priceEarningsRatio", "priceToBookRatio", "enterpriseValueMultiple", "priceToSalesRatio", "dividendYield", "priceEarningsToGrowthRatio"]

    df = df.loc[df['Ratio'].isin(our_ratios)]
    return df
    
#test = financialratios('TSLA')  
#test


In [3]:
# comp_stocks = tickers_adj

def comparison_ratios(target_stock, comp_stocks):
    
    x = financialratios(target_stock)
    
    for stock in comp_stocks:
        try:
            y = financialratios(stock)
            x = x.merge(y, on='Ratio')
            
        # Can't get data for x stocks so added in an error handling try except block to get around this.    
        except KeyError: 
            print("KeyError, cannot get data for", stock) 
    
    return x

# comparison_ratios('TSLA', comp_stocks)

## Scraping The S&P 500 Tickers

In [4]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'class':'wikitable sortable'})
    stocks_comp = {}
    stocks_sector = {}
    
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('\n','')
        company = row.findAll('td')[1].text.replace('\n','')
        sector = row.findAll('td')[3].text.replace('\n','')
        if'.' in ticker:
            ticker = ticker.replace('.','.')
            print('ticker replaced to', ticker)
        stocks_comp[ticker] = company
        stocks_sector[ticker] = sector
    
    return stocks_comp, stocks_sector

stocks_func = save_sp500_tickers()
stocks_comp = stocks_func[0]
stocks_sector = stocks_func[1]
stocks_func

NameError: name 'requests' is not defined

In [ ]:
#Return a list of the tickers
comp_stock_tickers = list(stocks_comp.keys())
# comp_stock_tickers = comp_stock_tickers[:10]
#Remove the first 10 stocks filter when time to run the full set
# comp_stock_tickers

## Running The Comparison Function

In [ ]:
# df = comparison_ratios("TSLA", comp_stock_tickers)
# df

df = comparison_ratios("TSLA", comp_stock_tickers)


We couldn't get data on all of the S&P 500 companies are are missing data for Carrier Global Corp (CARR), Howmet Aerospace Inc (HWM) and Otis Worldwide Corp (OTIS). Given that this is only 3 companies, we will continue with our dataset.


## Cleaning Our Data



In [ ]:
#Transpose the data

df = df.set_index('Ratio').T.rename_axis(None,axis=1).reset_index()
df.head()

In [ ]:
df['company'] = df['index'].map(stocks_comp)
df['sector'] = df['index'].map(stocks_sector)
df.head()

In [ ]:
#column headings

col_headings=list(df.columns.values)
col_headings

In [ ]:
#change column names
df.columns = ['ticker',
 'price_to_book_ratio',
 'price_to_sales_ratio',
 'price_earnings_ratio',
 'price_earnings_to_growth_ratio',
 'dividend_yield',
 'enterprise_value_multiple',
 'company',
 'sector']
df.head()

In [ ]:
#rearrange columns
df = df[['ticker',
         'company',
 'sector',
 'price_to_book_ratio',
 'price_to_sales_ratio',
 'price_earnings_ratio',
 'price_earnings_to_growth_ratio',
 'dividend_yield',
 'enterprise_value_multiple']]

df.head()

In [ ]:
#We could use fillna() but am going to use iloc instead for practice.
#df['company'].fillna()

df.iloc[0, df.columns.get_loc('company')] = "Tesla"
df.iloc[0, df.columns.get_loc('sector')] = "Consumer Discretionary"
df.head()

## Save To CSV

In [ ]:
df.to_csv('company_fin_ratios.csv')

## Next Steps

Data visualisation with Plotly.